# Hiatuses

In [ ]:
import os
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - matplotlib.pyplot

In [ ]:
from paths import path_results, CESM_filename
from timeseries import IterateOutputCESM

In [ ]:
gmst_ctrl = xr.open_dataset(f'{path_results}/GMST/GMST_ctrl.nc')
gmst_rcp  = xr.open_dataset(f'{path_results}/GMST/GMST_rcp.nc' )

In [ ]:
lfit_ctrl = gmst_ctrl.lin_fit .attrs['lin_fit_params']
qfit_ctrl = gmst_ctrl.quad_fit.attrs['quad_fit_params']
lfit_rcp  = gmst_rcp .lin_fit .attrs['lin_fit_params']
qfit_rcp  = gmst_rcp .quad_fit.attrs['quad_fit_params']

In [ ]:
lfit_ctrl_trend = np.empty((len(gmst_ctrl.year)))
qfit_ctrl_trend = np.empty((len(gmst_ctrl.year)))
for i, t in enumerate(gmst_ctrl.year.values):  
    lfit_ctrl_trend[i] = lfit_ctrl[0]    
    qfit_ctrl_trend[i] = 2*qfit_ctrl[0]*t + qfit_ctrl[1]
    
lfit_rcp_trend  = np.empty((len(gmst_rcp.year)))
qfit_rcp_trend  = np.empty((len(gmst_rcp.year)))
for i, t in enumerate(gmst_rcp.year.values):  
    lfit_rcp_trend[i] = lfit_rcp[0]    
    qfit_rcp_trend[i] = 2*qfit_rcp[0]*i + qfit_rcp[1]

In [ ]:
f, ax = plt.subplots(2,2, figsize=(16,8), sharex='col')

for i in range(2):
    ax[1,i].axhline(0, c='k', lw=.5)
    ax[1,i].set_xlabel('time [years]', fontsize=16)
    for j in range(2):
        ax[i,j].tick_params(labelsize=14)
    ax[0,i].set_ylabel('GMST [$^\circ$C]', fontsize=16)
    ax[1,i].set_ylabel('trend [$^\circ$C/decade]', fontsize=16)

ax[0,0].plot(gmst_ctrl.year, gmst_ctrl.lin_fit , c='grey', ls='-' , label='lin. fit')
ax[0,0].plot(gmst_ctrl.year, gmst_ctrl.quad_fit, c='grey', ls='--', label='quad. fit')
ax[0,0].plot(gmst_ctrl.year, gmst_ctrl.GMST, lw=2, c='C0', label='')
ax[0,0].text(0.02, 0.9, 'CTRL', transform=ax[0,0].transAxes, fontsize=16)
ax[0,0].legend(loc=4, fontsize=14)


ax[1,0].plot(gmst_ctrl.year, lfit_ctrl_trend*10, c='grey', ls='-' )
ax[1,0].plot(gmst_ctrl.year, lfit_ctrl_trend*10-.04, c='g', ls='-' )
ax[1,0].plot(gmst_ctrl.year, lfit_ctrl_trend*10-.07, c='g', ls='-' )
ax[1,0].plot(gmst_ctrl.year, lfit_ctrl_trend*10-.17, c='g', ls='-' )
# ax[1,0].plot(gmst_ctrl.year, qfit_ctrl_trend*10, c='grey', ls='--')
# ax[1,0].plot(gmst_ctrl.year, qfit_ctrl_trend*10-.17, c='g', ls='--')
ax[1,0].plot(gmst_ctrl.year, gmst_ctrl.trend_10*10, lw=1, c='C0', ls=':' , label='10 years')
ax[1,0].plot(gmst_ctrl.year, gmst_ctrl.trend_15*10, lw=2, c='C0', ls='-' , label='15 years')
ax[1,0].plot(gmst_ctrl.year, gmst_ctrl.trend_30*10, lw=1, c='C0', ls='--', label='30 years')
ax[1,0].legend(fontsize=14, title='window size')

ax[0,1].plot(gmst_rcp.year, gmst_rcp.lin_fit , c='grey', ls='-' )
ax[0,1].plot(gmst_rcp.year, gmst_rcp.quad_fit, c='grey', ls='--')
ax[0,1].plot(gmst_rcp.year, gmst_rcp.GMST, lw=2, c='C1')
ax[0,1].text(0.02, 0.9, 'RCP', transform=ax[0,1].transAxes, fontsize=16)

ax[1,1].plot(gmst_rcp.year, lfit_rcp_trend*10, c='grey', ls='-' )
ax[1,1].plot(gmst_rcp.year, qfit_rcp_trend*10, c='grey', ls='--')
ax[1,1].plot(gmst_rcp.year, qfit_rcp_trend*10-.04, c='g', ls='--')
ax[1,1].plot(gmst_rcp.year, qfit_rcp_trend*10-.07, c='g', ls='--')
ax[1,1].plot(gmst_rcp.year, qfit_rcp_trend*10-.17, c='g', ls='--')
ax[1,1].plot(gmst_rcp.year, gmst_rcp.trend_10 *10, lw=1, c='C1', ls=':' , label='10 years')
ax[1,1].plot(gmst_rcp.year, gmst_rcp.trend_15 *10, lw=2, c='C1', ls='-' , label='15 years')
ax[1,1].plot(gmst_rcp.year, gmst_rcp.trend_30 *10, lw=1, c='C1', ls='--', label='30 years')
ax[1,1].legend(fontsize=14, title='window size')

f.tight_layout()
f.align_ylabels()


which thresholds to choose (from Hedemann 2016)?
- 0.17: mismatch between CMIP5 ensemble mean
- 0.04-0.07: deviation from long term trend